In [28]:
import pandas as pd
import random

In [ ]:
rename
df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
# Or rename the existing DataFrame (rather than creating a copy) 
df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)


### Data clensing

In [267]:
# load file
df = pd.read_excel('test.xlsm')
#df=pd.DataFrame(df)
#df[df['Unnamed: 2']==4]  # select Unnamed: 2 value ==4 
# rename columns and index
df = df.rename(columns={df.columns[0]:'Name',
                        df.columns[1]:'Code',
                        df.columns[2]:'Type',
                        df.columns[3]:'Holiday',
                        df.columns[4]:'Weekday'})
df = df.rename(index={0:'Weekday_ch',1:'Weekday_num',2:'is_holiday'})
# unselect unnamned coluns
unselect_unnamed = [col for col in df if 'Unnamed' not in str(col)]  # 注意 有些還是 date type
df = df[unselect_unnamed]

# rename date index from 1 to date
for i in range(5,len(df.columns)):
    df = df.rename(columns={df.columns[i]:str(i-4)}) 

df

,Name,Code,Type,Holiday,Weekday,1,2,3,4,5,...,22,23,24,25,26,27,28,29,30,31
Weekday_ch,8,月,NaN,NaN,NaN,六,日,一,二,三,...,六,日,一,二,三,四,五,六,日,一
Weekday_num,NaN,NaN,NaN,NaN,NaN,6,7,1,2,3,...,6,7,1,2,3,4,5,6,7,1
is_holiday,NaN,NaN,NaN,NaN,NaN,v,v,NaN,NaN,NaN,...,v,v,NaN,NaN,NaN,NaN,NaN,v,v,NaN
3,NaN,簡,類,假,平,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,李志謙,A,4,2,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,戴維安,B,4,1,7,NaN,NaN,NaN,1,NaN,...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,吳嘉紘,C,4,1,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,x,NaN
7,郭昱,D,4,3,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [249]:
df.loc['is_holiday'].notnull().sum()

10

In [268]:
def count_days(df, duty_type):
    """
    input = dataframe after clensing
    output = if there's no day violation
    """
    TYPES_OF_DUTY = {1:'CT', 2:'MR', 3:'ER', 4:'CR'}

    # test for holiday
    num_of_holiday = df.loc['is_holiday'].notnull().sum()
    num_of_weekday = df.loc['Weekday_num'].notnull().sum() - num_of_holiday
    print(num_of_holiday, num_of_weekday)
    
    num_of_duties_h = df[df['Type']==duty_type]['Holiday'].sum()
    num_of_duties_w = df[df['Type']==duty_type]['Weekday'].sum()
    
    # 若值班總數不足， raise error
    if num_of_duties_h < num_of_holiday:
#        print(f'{TYPES_OF_DUTY[duty_type]}班假日值班總數不足')
        raise ValueError(f'{TYPES_OF_DUTY[duty_type]}班假日值班總數不足，缺少{num_of_holiday-num_of_duties_h}班')
    if num_of_duties_w < num_of_weekday:
#        print(f'{TYPES_OF_DUTY[duty_type]}班平日值班總數不足')
        raise ValueError(f'{TYPES_OF_DUTY[duty_type]}班平日值班總數不足，缺少{num_of_weekday-num_of_duties_w}班')

    

        

In [269]:
count_days(df,4)

10 21


ValueError: CR班假日值班總數不足，缺少3班

In [234]:
df

,Name,Code,Type,Holiday,Weekday,1,2,3,4,5,...,22,23,24,25,26,27,28,29,30,31
Weekday_ch,8,月,NaN,NaN,NaN,六,日,一,二,三,...,六,日,一,二,三,四,五,六,日,一
Weekday_num,NaN,NaN,NaN,NaN,NaN,6,7,1,2,3,...,6,7,1,2,3,4,5,6,7,1
is_holiday,NaN,NaN,NaN,NaN,NaN,v,v,NaN,NaN,NaN,...,v,v,NaN,NaN,NaN,NaN,NaN,v,v,NaN
3,NaN,簡,類,假,平,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,李志謙,A,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,戴維安,B,4,3,5,NaN,NaN,NaN,1,NaN,...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,吳嘉紘,C,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,x,NaN
7,郭昱,D,4,3,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
unselect_unnamed = [str(col) for col in df if 'Unnamed' not in str(col) ]
unselect_unnamed
#df2 = df[unselect_unnamed]

['Name',
 'Code',
 'Type',
 'Holiday',
 'Weekday',
 '1900-01-01 00:00:00',
 '1900-01-02 00:00:00',
 '1900-01-03 00:00:00',
 '1900-01-05 00:00:00',
 '1900-01-06 00:00:00',
 '1900-01-07 00:00:00',
 '1900-01-08 00:00:00',
 '1900-01-09 00:00:00',
 '1900-01-10 00:00:00',
 '1900-01-11 00:00:00',
 '1900-01-12 00:00:00',
 '1900-01-13 00:00:00',
 '1900-01-14 00:00:00',
 '1900-01-15 00:00:00',
 '1900-01-16 00:00:00',
 '1900-01-17 00:00:00',
 '1900-01-18 00:00:00',
 '1900-01-19 00:00:00',
 '1900-01-20 00:00:00',
 '1900-01-21 00:00:00',
 '1900-01-22 00:00:00',
 '1900-01-23 00:00:00',
 '1900-01-24 00:00:00',
 '1900-01-25 00:00:00',
 '1900-01-26 00:00:00',
 '1900-01-27 00:00:00',
 '1900-01-28 00:00:00',
 '1900-01-29 00:00:00',
 '1900-01-30 00:00:00']

In [135]:
for item in df:
    print(type(item))

<class 'int'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'str'>
<class 'str'>


In [97]:
df2 = df[:]
df2.index


RangeIndex(start=0, stop=8, step=1)

In [98]:
import pandas as pd

data = {'spike-2': [1,2,3], 'hey spke': [4,5,6], 'spiked-in': [7,8,9], 'no': [10,11,12]}
df = pd.DataFrame(data)

spike_cols = [col for col in df.columns if 'spike' in col]
print(list(df.columns))
print(spike_cols)

['spike-2', 'hey spke', 'spiked-in', 'no']
['spike-2', 'spiked-in']


In [50]:
type(df2.columns[1])

str

In [49]:
for i in range(len(df2.columns)):
    if 'Unnamed' not in df2.columns[i]:
        print(y)

TypeError: argument of type 'int' is not iterable